In [1]:
import requests
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from credentials_v2 import username, token

In [4]:
# host_url = "https://qlued.alqor.io/"
device_name = "alqor_mot_simulator"
host_url = "http://localhost:8000/"
device_name = "localtest_mot_simulator"
url_prefix = host_url + f"api/v2/{device_name}/"

In [5]:
url = url_prefix + "get_config"
pprint(url)
r = requests.get(url, params={"username": username, "token": token})
r_dict = json.loads(r.text)
pprint(r_dict)

'http://localhost:8000/api/v2/alqor_mot_simulator/get_config'


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
job_payload = {
    "experiment_0": {
        "instructions": [
            ("load", [0], [20]),
            ("measure", [0], []),
        ],
        "num_wires": 1,
        "shots": 2,
        "wire_order": "interleaved",
    },
}

In [11]:
url = url_prefix + "post_job"

pprint(url)
job_response = requests.post(
    url,
    json={"job": json.dumps(job_payload), "username": username, "token": token},
)
job_id = (job_response.json())["job_id"]
print(job_response.json())

'https://qlued.alqor.io/api/v2/alqor_mot_simulator/post_job'
{'job_id': 'b19916f7429f418db1910af8', 'status': 'INITIALIZING', 'detail': 'Got your json.', 'error_message': 'None'}


In [12]:
url = url_prefix + "get_job_status"

status_response = requests.get(
    url, params={"job_id": job_id, "username": username, "token": token}
)

print(status_response.text)

{"job_id": "b19916f7429f418db1910af8", "status": "DONE", "detail": "None; Passed json sanity check; Compilation done. Shots sent to solver.", "error_message": "None"}


In [11]:
url = url_prefix + "get_job_result"

result_response = requests.get(
    url, params={"job_id": job_id, "username": username, "token": token}
)

pprint(result_response.json())

{'backend_name': 'alqor_mot_simulator',
 'backend_version': '0.1',
 'display_name': 'mot',
 'header': {},
 'job_id': 'b71e9a39f5fa4dcf836ae2d3',
 'qobj_id': None,
 'results': [{'data': {'memory': ['20', '20']},
              'header': {'extra metadata': 'text', 'name': 'experiment_0'},
              'shots': 2,
              'success': True}],
 'status': 'finished',
 'success': True}
